### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

**Question 1:** Set the multi-level index so the first level is the stock symbol itself, and the second level is the date.  Make sure the date column is sorted in ascending order.  You might have to use the `sort_index(level=0)` method to get the values straight.

**Important:** Make sure you use the `parse_dates` argument to read the date column as a date.  If you don't it might end up as a string.

In [53]:
import pandas as pd
stocks = pd.read_csv(r'C:\Users\iulia\OneDrive\Documents\Data Science\stocks_panel.csv', parse_dates=['Date'])
stocks = stocks.set_index(['Stock','Date']).sort_index(level=0)

In [54]:
stocks

Price
Stock Date                  
AAPL  2014-11-05  108.860001
      2014-11-06  108.699997
      2014-11-07  109.010002
      2014-11-10  108.830002
      2014-11-11  109.699997
...                      ...
MSFT  2019-10-28  144.190002
      2019-10-29  142.830002
      2019-10-30  144.610001
      2019-10-31  143.369995
      2019-11-01  143.720001

[6285 rows x 1 columns]

**Question 2:** To capture some other aspects of dates, create columns in your dataset that capture the following aspects of each timestamp:

  - What quarter it's in
  - Whether or not it's the last day of the month/quarter
  - What day it is (ie, do price changes vary by day?)
  
**Hint:** You don't use the `dt` attribute to get date parts from index values.  Multi indices are also a little tricky.  

To get what you want, try this: `df.index.get_level_values(level=1).your_datepart_here`

If you are looking for a refresher on different pandas date parts, you can find them here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [58]:
stocks['what_quarter']=stocks.index.get_level_values(level=1).quarter

In [61]:
stocks['is_month_end']=stocks.index.get_level_values(level=1).is_month_end

In [65]:
stocks['what_day']=stocks.index.get_level_values(level=1).day

In [66]:
stocks

Price  what_quarter  is_month_end  what_day
Stock Date                                                        
AAPL  2014-11-05  108.860001             4         False         5
      2014-11-06  108.699997             4         False         6
      2014-11-07  109.010002             4         False         7
      2014-11-10  108.830002             4         False        10
      2014-11-11  109.699997             4         False        11
...                      ...           ...           ...       ...
MSFT  2019-10-28  144.190002             4         False        28
      2019-10-29  142.830002             4         False        29
      2019-10-30  144.610001             4         False        30
      2019-10-31  143.369995             4          True        31
      2019-11-01  143.720001             4         False         1

[6285 rows x 4 columns]

**Question 3:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

However, your data won't really "know" about those values unless they can be observed alongside the current observation.  Data is read in as rows, not columns.  

To that end, make two columns that capture the value of the following:

 - What the previous recorded price for each stock was
 - The stock price from two observations ago
 
**Remember:** This has to be done on a particular level of the index so make sure it's getting applied appropriately!

In [71]:
stocks['previous_price']=stocks.groupby(level=0)['Price'].shift()

In [73]:
stocks['2_observations_ago']=stocks.groupby(level=0)['Price'].shift(2)

**Question 4:** How did each stock price change compared to the S&P 500? 

Lots of times it's useful to see how something else moves with some other item that you're trying to track.  

In the data folder is a file called `s&p.csv`, and it contains the price history of the S&P 500 index for each day since its inception. See if you can upload it, and merge the `adj close` column into your dataset, so there's a column that displays the observed value of the index for every single price observation we have in our dataset.

**Hints:**
 - Merging on multi-level indices is tricky and prone to failure.  To make this a little bit easier, just use `reset_index()` to pop out the date column in the multi-index, and merge on it as if it were a regular column.
 - Make sure both date columns are actually encoded as dates, rather than strings, or else the merge won't work.
 - You'll want to go back to the multi-level index when you're done with this step.

In [100]:
sp = pd.read_csv('../../Data/s&p.csv')

In [131]:
sp['Adj Close'].mean()

465.89700957805263

In [119]:
comapare = stocks.merge(sp,left_index= True, right_index=True)

In [128]:
comapare.groupby(level =0).mean()[['Price','Adj Close']]

,Price,Adj Close
Stock,,
AAPL,149.512617,2418.212274
AMZN,1069.032395,2418.212274
FB,139.225958,2418.212274
GOOGL,902.415203,2418.212274
MSFT,77.800096,2418.212274


**Question 5:** Window Statistics

Lots of times, we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [9]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.head()

,Value
2020-01-01,-0.253379
2020-01-02,-0.838158
2020-01-03,-1.131807
2020-01-04,-1.708901
2020-01-05,-0.196300


In [11]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,-0.366059


You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 5 & 10 day moving averages for each stock inside the dataset.

**Note:** Do *not* try and merge them back into your dataset yet, just make sure you have the values showing up.

And be mindful of performing these on the appropriate levels of your dataset.

In [ ]:
# your answer here

If you take a look at the index, you should notice that it has *three* levels to it, and not just two like before.  

Combining datasets with differing numbers of levels is cumbersome, and there's a decent amount of churn in what methods work from one version of Pandas to another.  

For now, try and get these values back into your original dataset by taking the following steps:

 - calculate the 5 & 10 day rolling averages for each stock price on the multilevel index, and and then use the `values` attribute for each one to drop the index and just get the column values (ask me about this if you have questions).  Your results should be a numpy array.
 - If you get an error message about unequal indexes, you can use the `reset_index()` method on your stocks data to get out of the multi-level index
 - use the results from the previous step to create new columns for each array
 
So as a quick example, it would sort of work like this:

`five_day = df.groupby(level=0)['Price'].your_stuff_here.values`

And then use this as the basis to make your new column from your original dataframe that has its index reset.

In [ ]:
# your answer here